This notebook cleans .txt and .dta data for students performance on exams for BIO1111 and exports a csv which can be analyzed in an R script.
#### Author: Christopher Agard

In [1]:
import pandas as pd
import numpy as np
import os,glob
from soar import *

pd.options.display.max_columns = 50

# Setting source and output folders

In [2]:
tu={'worksource18_1': "C:/Users/tuh27554/Documents/BIOL1111/Fall 2018/Raw Data/Exam 1",
    'worksource19_1': "C:/Users/tuh27554/Documents/Grading/2019/FALL EX1/Input Data",
    'workoutput': "C:/Users/tuh27554/Documents/Grading/2019/Output"}
# gandolf={'worksource': "C:/Users/craga/Google Drive/TU 2018/Fall/Exam 1/Input data",
#     'workoutput': "C:/Users/craga/Google Drive/TU 2018/Fall/Exam 1/Output Data"}

source = tu

# Getting and Cleaning Exam Data

In [3]:
os.chdir(source['worksource'])
files = glob.glob('*.txt')
files

['V1Raw.txt', 'V2Raw.txt']

In [6]:
os.chdir(source['worksource'])
df1=cleanSoarExam(files[0],examNum=1,soarSessions=[81])
df1['version'] = 1
df2=cleanSoarExam(files[1],examNum=1,soarSessions=[81])
df2['version'] = 2
df = df1.append(df2)
df.head()

C:\Users\tuh27554\Documents\GitHub\Grading\soar.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  tmp['soar'] = soar


,tuid,last,first,middle,unnamed1,unnamed2,unnamed3,soar,ncorrect,examNumber,item_1,item_2,item_3,item_4,item_5,item_6,item_7,item_8,item_9,item_10,item_11,item_12,item_13,item_14,item_15,item_16,item_17,item_18,item_19,item_20,item_21,item_22,item_23,item_24,item_25,item_26,item_27,soarType,version
0,NNNNNNNNN,NNNNNNNNNNN,NNNNNN,N,NaN,NaN,NaN,NaN,27,1,2,4,2,2,3,1,4,2,3,2,3,4,4,3,1,3,1,1,1,3,2,3,2,1,3,1,4,key,1
1,915552082,RAMASWAMY,AKSHAY,A,11.0,10.0,80.0,72.0,19,1,2,4,3,4,3,1,4,1,3,2,3,4,4,3,3,3,1,1,1,2,1,3,2,1,3,3,1,other,1
2,915425371,PETERSON,TARYAN,K,11.0,10.0,49.0,87.0,15,1,2,4,2,2,3,3,3,1,4,3,2,4,4,3,2,3,1,1,1,3,1,2,3,1,3,4,2,other,1
3,915522412,GHOSH,KEYA,NaN,11.0,10.0,44.0,87.0,12,1,3,4,4,2,3,1,2,4,1,4,3,4,3,1,4,2,1,1,1,1,1,4,3,1,3,2,4,other,1
4,915552190,EBA,TASNIM,NaN,11.0,10.0,14.0,72.0,21,1,2,4,2,2,3,1,4,3,2,2,3,3,4,3,1,3,2,1,1,3,1,3,2,1,3,1,2,other,1


We will confirm that the exam key have been appropriately identified and change it if necessary.

Now we change the dir to the outputfolder and save the cleaned, merged file.

In [7]:
itemcols = df.columns[df.columns.str.contains('item')]
itemcols

Index(['item_1', 'item_2', 'item_3', 'item_4', 'item_5', 'item_6', 'item_7',
       'item_8', 'item_9', 'item_10', 'item_11', 'item_12', 'item_13',
       'item_14', 'item_15', 'item_16', 'item_17', 'item_18', 'item_19',
       'item_20', 'item_21', 'item_22', 'item_23', 'item_24', 'item_25',
       'item_26', 'item_27'],
      dtype='object')

In [8]:
for col in itemcols:
    df[col] = df[col].apply(letters2num)

In [9]:
os.chdir(source['workoutput'])
df.to_csv("Fall18Exam 1_cleaned.csv")

# Analyzing Exam Data

1. [Most Frequently Wrong](#Most-Frequently-Wrong-Answers)

## Strange values for items

In reviewing the data I noted some values outside of the expected 1-4.  Could you help me understand what these are?

In [10]:
all_responses = pd.concat([pd.Series(df[col].unique()) for col in itemcols]).unique()
strange_response = [resp for resp in all_responses if resp not in ['A','B','C','D','E','1','2','3','4','5']]
strange_resp_item=[col for col in itemcols if df.loc[df[col].isin(strange_response),col].shape[0]>0]
print("The strange responses {} were found for the following items:\n{}".format(strange_response, strange_resp_item))


The strange responses [' ', nan] were found for the following items:
['item_7', 'item_15', 'item_17', 'item_20', 'item_24', 'item_25', 'item_26', 'item_27']


# Evaluating Student Performance

## Score distribution
Now we look at scores by section. 

In [13]:
df.ncorrect.describe()

count    268.000000
mean      19.873134
std        4.403644
min        7.000000
25%       17.000000
50%       20.000000
75%       23.000000
max       27.000000
Name: ncorrect, dtype: float64

In [14]:
df.groupby(['soar']).ncorrect.describe()

,count,mean,std,min,25%,50%,75%,max
soar,,,,,,,,
2.0,2.0,23.000000,5.656854,19.0,21.00,23.0,25.00,27.0
6.0,2.0,21.000000,8.485281,15.0,18.00,21.0,24.00,27.0
21.0,2.0,19.500000,10.606602,12.0,15.75,19.5,23.25,27.0
71.0,31.0,19.064516,4.972829,7.0,16.00,19.0,22.50,27.0
72.0,34.0,19.676471,4.662686,9.0,17.00,21.0,23.00,27.0
73.0,34.0,21.088235,3.824702,11.0,19.00,22.0,23.00,27.0
75.0,35.0,18.371429,4.485832,9.0,15.50,19.0,20.50,27.0
81.0,33.0,20.575758,3.832675,12.0,18.00,21.0,23.00,27.0
83.0,28.0,19.392857,4.382958,12.0,15.75,20.0,22.00,27.0


## Item Analysis
1. [Percent choice for each item](#Percent-choice-for-each-item)

In [15]:
os.chdir(source['worksource'])
dfcomp = pd.read_csv('Cross-Walk 2019 Fall to 2018 Fall Exams.csv')
dfcomp

,F19E1,F18E1
0,1,1
1,2,2
2,3,4
3,4,5
4,5,3
5,6,6
6,7,10
7,8,7
8,9,8
9,10,9


### Percent choice for each item

In [17]:
df_item1 = pd.DataFrame()
for col in itemcols:
    df_item1 = df_item1.append(df.loc[df.version==1,col].value_counts(normalize=True))
df_item1 = df_item1.applymap(lambda x: "{:.0f}".format(x*100))
df_item1 = df_item1.reset_index().rename(columns={'index':'F18E1'})
df_item1['F18E1'] = df_item1.F18E1.apply(lambda x: int(x[5:]))
# df_item1 = df_item1.sort_values('F18E1')
# df_item1['F19E1'] = dfcomp.F18E1
df_item1.to_csv('Fall 2018v1.csv')
df_item1

,F18E1,A,B,C,D,
0,1,8,1,90,1,nan
1,2,93,5,2,nan,nan
2,3,1,26,71,1,nan
3,4,10,3,83,4,nan
4,5,1,83,7,9,nan
5,6,11,7,12,71,nan
6,7,76,16,2,4,1
7,8,21,7,64,8,nan
8,9,10,56,29,4,nan
9,10,6,10,83,1,nan


In [18]:
df_item = pd.DataFrame()
for col in itemcols:
    df_item = df_item.append(df.loc[df.version==2,col].value_counts(normalize=True))
df_item = df_item.applymap(lambda x: "{:.0f}".format(x*100))
df_item

,A,B,C,D,
item_1,4,91,4,1,nan
item_2,95,1,3,1,nan
item_3,nan,73,27,nan,nan
item_4,1,82,6,10,nan
item_5,10,5,80,5,nan
item_6,40,5,52,3,nan
item_7,3,19,77,1,nan
item_8,6,3,20,71,nan
item_9,27,9,59,6,nan
item_10,10,50,35,5,1
